# Optimal Storage for average years

This notebook performs the main simulations of the paper. For three countries -
France, Germany, and Greece - we simulate the optimal share of wind and solar
power given the share of renewable generation in total annual generation.

## Packages and options

In [1]:
# add the main directory to the path
import sys
sys.path.append("..")

import logging
import pandas as pd
import numpy as np
import tqdm

from model import simulate_country_averages, calculate_storage_need, get_average_profiles

logging.basicConfig(level=logging.WARNING)

In [2]:
fn_renewable: str = "../data/renewables_ninja.parquet"
fn_demand: str = "../data/renewables_with_load.parquet"

map_countries = {
    "FR": "France",
    "DE": "Germany",
    "GR": "Greece",
}
countries = list(map_countries.keys())
years = range(2015, 2020)
# TODO DO NOT CHANGE NEXT LINE
total_demand = 100 # DO NOT CHANGE THIS ==> ENSURES % interpretation
steps = 100 # number of steps for renewable shares

## Simulations

### Optimal RE Shares

This simulation derives the optimal wind share given the share of renewable
generation in total generation. It is performed for each renewable share between
zero and 100%.

In [3]:
shares_renewable = list(np.linspace(0, 1, steps + 1))
df_opt = simulate_country_averages(
    countries=countries,
    shares_renewable=shares_renewable,
    total_demand=total_demand,
    years=years,
    fn_demand=fn_demand,
    fn_renewable=fn_renewable
)

  0%|          | 0/303 [00:00<?, ?it/s]

100%|██████████| 303/303 [05:56<00:00,  1.18s/it]

Solve 303 scenarios in : 356.23 seconds.
Failed to solve 0 of 303 scenarios


### Given RE Shares


In [4]:
share_wind = 0.5
lst_df = []
all_combis = [(c, s) for c in countries for s in shares_renewable]
for country, share_renewable in tqdm.tqdm(all_combis):
    df_profiles = get_average_profiles(
        country=country, years=years, fn_demand=fn_demand, fn_renewable=fn_renewable)
    df_ = calculate_storage_need(
            df_profiles=df_profiles, 
            share_renewable=share_renewable, 
            share_wind=share_wind, 
            total_demand=total_demand
        ).assign(
                country=country,
                yearWeather="average",
                yearDemand="average",
                renewableDemandShare=share_renewable,
            )
    lst_df.append(df_)
df_given = pd.concat(lst_df)
# df_given

100%|██████████| 303/303 [05:29<00:00,  1.09s/it]


In [5]:
df_given.to_parquet("../results/df_storage_need_given.parquet")
df_opt.to_parquet("../results/df_storage_need_opt.parquet")